<a href="https://colab.research.google.com/github/GezhinOleg/Num_Pnd_L4/blob/main/ds_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

    если источник traffic_source равен yandex или google, то в source_type ставится organic
    для источников paid и email из России - ставим ad
    для источников paid и email не из России - ставим other
    все остальные варианты берем из traffic_source без изменений

In [ ]:
import pandas as pd

In [ ]:
log = pd.read_csv('datas/visit_log.csv', sep=';')

In [ ]:
log.loc[log['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'
log.loc[log['traffic_source'].isin(['paid', 'email']) & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[log['traffic_source'].isin(['paid', 'email']) & (log.region != 'Russia'), 'source_type'] = 'other'
log.loc[log.source_type.isnull(), 'source_type']=log['traffic_source']
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

    Прочитайте содержимое файла с датафрейм
    Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
import pandas as pd
import re

In [ ]:
urls = pd.read_csv('datas/URLs.txt')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
urls[urls.url.str.contains('/[0-9]{8}-', regex = True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
rating_count = ratings.groupby('userId').count().reset_index()
rating_count = rating_count[['userId', 'rating']]

In [ ]:
count_rating_100 = rating_count[rating_count['rating']>100]
count_rating_100.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [ ]:
rating_max=ratings.groupby('userId').max().reset_index()
rating_min=ratings.groupby('userId').min().reset_index()

In [ ]:
rating_max_timestamp=rating_max[['userId','timestamp']]
rating_min_timestamp=rating_min[['userId','timestamp']]

In [ ]:
rating_max_timestamp.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
rating_min_timestamp.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

C:\Users\Admin1\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
rating_max_timestamp.head()

,userId,timestamp_max
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [ ]:
rating_min_timestamp.head()

,userId,timestamp_min
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [ ]:
avg_timestamp=count_rating_100.merge(rating_max_timestamp, how='inner').merge(rating_min_timestamp, how='inner')

In [ ]:
def average(row):
    mean_=(row['timestamp_max']-row['timestamp_min'])
    return mean_

In [ ]:
avg_timestamp['avg']=avg_timestamp.apply(average, axis=1)
len(count_rating_100)==len(avg_timestamp)

True

In [ ]:
avg_timestamp.head(15)

,userId,rating,timestamp_max,timestamp_min,avg
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282
5,21,162,854522908,853157476,1365432
6,22,220,1131753381,1131661890,91491
7,23,726,1166728253,1148386124,18342129
8,26,172,1371811577,1351544316,20267261
9,30,1011,1060795346,944943070,115852276


## Задание 4

    Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
    Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
table=rzd.merge(auto, how='outer', on='client_id')

In [ ]:
table=table.merge(air, how='outer', on='client_id')

In [ ]:
table.loc[table.rzd_revenue.isnull(), 'rzd_revenue',]=0
table.loc[table.auto_revenue.isnull(), 'auto_revenue',]=0
table.loc[table.air_revenue.isnull(), 'air_revenue',]=0

In [ ]:
full_table=table.merge(client_base, how='outer', on='client_id')

In [ ]:
table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [ ]:
full_table

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
